### 01 - Setup and test

In [1]:
from pyspark.sql import SparkSession
import os

# -----------------------------
# Env
# -----------------------------
# Polaris
POLARIS_URI = os.getenv("POLARIS_URI", "http://polaris:8181/api/catalog").rstrip("/")
POLARIS_OAUTH2 = os.getenv("POLARIS_OAUTH2_TOKEN_URL", "http://polaris:8181/api/catalog/v1/oauth/tokens")
POLARIS_SCOPE = os.getenv("POLARIS_SCOPE", "PRINCIPAL_ROLE:ALL")
POLARIS_CLIENT_ID = os.getenv("POLARIS_CLIENT_ID", "admin")
POLARIS_CLIENT_SECRET = os.getenv("POLARIS_CLIENT_SECRET", "password")

# Spark
SPARK_MASTER = os.getenv("SPARK_MASTER", "spark://spark-master:7077")
DRIVER_HOST = os.getenv("SPARK_DRIVER_HOST", "jupyter")  # docker service name

# MinIO / S3A
S3_ENDPOINT = os.getenv("S3_ENDPOINT", "http://minio:9000")
S3_ACCESS_KEY = os.getenv("MINIO_ROOT_USER", "minioadmin")
S3_SECRET_KEY = os.getenv("MINIO_ROOT_PASSWORD", "minioadmin")

# Polaris REST "warehouse" = warehouse identifier (meestal catalog name), GEEN s3:// pad
POLARIS_WAREHOUSE = os.getenv("POLARIS_WAREHOUSE", "polaris").strip()

# AWS region (nodig voor Iceberg S3FileIO op executors)
AWS_REGION = os.getenv("AWS_REGION", "us-east-1").strip()

# -----------------------------
# Stop oude sessie
# -----------------------------
if "spark" in locals():
    try:
        spark.stop()
    except Exception:
        pass

print(f"🔗 SPARK_MASTER       : {SPARK_MASTER}")
print(f"🧷 DRIVER_HOST        : {DRIVER_HOST}")
print(f"🧭 POLARIS_URI        : {POLARIS_URI}")
print(f"🏷️  POLARIS_WAREHOUSE  : {POLARIS_WAREHOUSE}")
print(f"🪣 S3 endpoint        : {S3_ENDPOINT}")
print(f"🌍 AWS_REGION         : {AWS_REGION}")

# -----------------------------
# SparkSession
# -----------------------------
builder = (
    SparkSession.builder
    .appName("Lakehouse-Unplugged")
    .master(SPARK_MASTER)
    .config("spark.driver.host", DRIVER_HOST)
    .config("spark.driver.bindAddress", "0.0.0.0")
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer")

    # Iceberg
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")

    # Maak Polaris de default catalog (dan is 'bronze.table' genoeg, maar polaris.bronze.table mag ook)
    .config("spark.sql.defaultCatalog", "polaris")

    # Polaris catalog
    .config("spark.sql.catalog.polaris", "org.apache.iceberg.spark.SparkCatalog")
    .config("spark.sql.catalog.polaris.type", "rest")
    .config("spark.sql.catalog.polaris.uri", POLARIS_URI)
    .config("spark.sql.catalog.polaris.warehouse", POLARIS_WAREHOUSE)

    # OAuth
    .config("spark.sql.catalog.polaris.rest.auth.type", "oauth2")
    .config("spark.sql.catalog.polaris.credential", f"{POLARIS_CLIENT_ID}:{POLARIS_CLIENT_SECRET}")
    .config("spark.sql.catalog.polaris.oauth2-server-uri", POLARIS_OAUTH2)
    .config("spark.sql.catalog.polaris.scope", POLARIS_SCOPE)
    .config("spark.sql.catalog.polaris.token-refresh-enabled", "true")

    # Iceberg FileIO via AWS bundle (s3:// naar MinIO)
    .config("spark.sql.catalog.polaris.io-impl", "org.apache.iceberg.aws.s3.S3FileIO")
    .config("spark.sql.catalog.polaris.s3.endpoint", S3_ENDPOINT)
    .config("spark.sql.catalog.polaris.s3.path-style-access", "true")
    .config("spark.sql.catalog.polaris.s3.access-key-id", S3_ACCESS_KEY)
    .config("spark.sql.catalog.polaris.s3.secret-access-key", S3_SECRET_KEY)
    .config("spark.sql.catalog.polaris.s3.region", AWS_REGION)

    # 🔥 Belangrijk: region beschikbaar maken voor executors/driver (AWS SDK v2)
    .config("spark.executorEnv.AWS_REGION", AWS_REGION)
    .config("spark.executorEnv.AWS_DEFAULT_REGION", AWS_REGION)
    .config("spark.driverEnv.AWS_REGION", AWS_REGION)
    .config("spark.driverEnv.AWS_DEFAULT_REGION", AWS_REGION)

    # S3A voor landing reads (s3a://...)
    .config("spark.hadoop.fs.s3a.endpoint", S3_ENDPOINT)
    .config("spark.hadoop.fs.s3a.access.key", S3_ACCESS_KEY)
    .config("spark.hadoop.fs.s3a.secret.key", S3_SECRET_KEY)
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .config("spark.hadoop.fs.s3a.endpoint.region", AWS_REGION)

    .config("spark.sql.shuffle.partitions", "4")
    .config("spark.sql.adaptive.enabled", "true")
)

pkgs = (os.getenv("SPARK_JARS_PACKAGES") or "").strip()
if pkgs:
    print(f"⚠️ spark.jars.packages staat aan (driver downloadt deps): {pkgs}")
    builder = builder.config("spark.jars.packages", pkgs)

spark = builder.getOrCreate()

print("✅ Spark up.")
print("defaultCatalog =", spark.conf.get("spark.sql.defaultCatalog"))
print("polaris.warehouse =", spark.conf.get("spark.sql.catalog.polaris.warehouse"))
print("AWS_REGION (driver env) =", os.getenv("AWS_REGION"))
print("🧪 Sanity spark.range(10).count() =", spark.range(10).count())


🔗 SPARK_MASTER       : spark://spark-master:7077
🧷 DRIVER_HOST        : jupyter
🧭 POLARIS_URI        : http://polaris:8181/api/catalog
🏷️  POLARIS_WAREHOUSE  : polaris
🪣 S3 endpoint        : http://minio:9000
🌍 AWS_REGION         : us-east-1


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/07 12:29:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


✅ Spark up.
defaultCatalog = polaris
polaris.warehouse = polaris
AWS_REGION (driver env) = None


🧪 Sanity spark.range(10).count() = 10


In [2]:
spark.sql("CREATE NAMESPACE IF NOT EXISTS bronze")
spark.sql("SHOW NAMESPACES").show()


+---------+
|namespace|
+---------+
|   bronze|
+---------+



In [3]:
spark.sql("""
CREATE TABLE IF NOT EXISTS bronze.test_table (
  id INT,
  txt STRING
) USING iceberg
""")

spark.sql("SHOW TABLES IN bronze").show()


+---------+----------+-----------+
|namespace| tableName|isTemporary|
+---------+----------+-----------+
|   bronze|test_table|      false|
+---------+----------+-----------+



#### 02 - Parkeer bestande in de landingzone

In [4]:
import os
import boto3
from pathlib import Path
from botocore.exceptions import ClientError

# ======================================================================
# 0️⃣ Helper: zoek automatisch lokaal data-bestand
# ======================================================================
def find_data_file(filename: str) -> Path:
    p = Path.cwd()
    for _ in range(6):
        candidate = p / "data" / filename
        if candidate.exists():
            return candidate
        p = p.parent
    raise FileNotFoundError(f"❌ Kon '{filename}' niet vinden in een 'data' map vanaf {Path.cwd()}.")

# ======================================================================
# 1️⃣ Config (uit env waar kan)
# ======================================================================
local_file = find_data_file("gekentekendevoertuigen_sample.json")

bucket = os.getenv("MINIO_BUCKET", "warehouse")
prefix = os.getenv("MINIO_PREFIX", "landing")

endpoint = os.getenv("S3_ENDPOINT", "http://minio:9000")
access_key = os.getenv("MINIO_ROOT_USER", "minioadmin")
secret_key = os.getenv("MINIO_ROOT_PASSWORD", "minioadmin")
region = os.getenv("AWS_REGION", "us-east-1")

object_key = f"{prefix}/{local_file.name}"
s3a_uri = f"s3a://{bucket}/{object_key}"

print(f"📄 Lokaal bestand : {local_file}")
print(f"⬆️ Upload naar    : s3://{bucket}/{object_key}")
print(f"📥 Spark read via : {s3a_uri}")
print(f"🪣 MinIO endpoint : {endpoint}")

# ======================================================================
# 2️⃣ MinIO client via boto3 (S3 API)
# ======================================================================
s3 = boto3.client(
    "s3",
    endpoint_url=endpoint,
    aws_access_key_id=access_key,
    aws_secret_access_key=secret_key,
    region_name=region,
)

# Bucket check (minio-setup maakt 'warehouse' al aan, maar dit maakt het robuust)
try:
    s3.head_bucket(Bucket=bucket)
except ClientError:
    print(f"ℹ️ Bucket '{bucket}' bestaat nog niet, maak 'm aan...")
    s3.create_bucket(Bucket=bucket)

# Upload bestand
s3.upload_file(str(local_file), bucket, object_key)
print("✅ Upload gelukt.")

# ======================================================================
# 3️⃣ Verify: lijst objecten in prefix
# ======================================================================
response = s3.list_objects_v2(Bucket=bucket, Prefix=prefix)
print("📦 Objecten in MinIO:")
for item in response.get("Contents", []):
    print(" -", item["Key"])

# ======================================================================
# 4️⃣ Spark read via S3A
# ======================================================================
df = spark.read.option("multiline", "true").json(s3a_uri)

print(f"📊 Aantal records: {df.count():,}")
df.printSchema()
df.show(5, truncate=False)


📄 Lokaal bestand : /workspace/data/gekentekendevoertuigen_sample.json
⬆️ Upload naar    : s3://warehouse/landing/gekentekendevoertuigen_sample.json
📥 Spark read via : s3a://warehouse/landing/gekentekendevoertuigen_sample.json
🪣 MinIO endpoint : http://minio:9000
✅ Upload gelukt.
📦 Objecten in MinIO:
 - landing/gekentekendevoertuigen_sample.json


26/01/07 12:29:48 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
26/01/07 12:29:48 WARN VersionInfoUtils: The AWS SDK for Java 1.x entered maintenance mode starting July 31, 2024 and will reach end of support on December 31, 2025. For more information, see https://aws.amazon.com/blogs/developer/the-aws-sdk-for-java-1-x-is-in-maintenance-mode-effective-july-31-2024/
You can print where on the file system the AWS SDK for Java 1.x core runtime is located by setting the AWS_JAVA_V1_PRINT_LOCATION environment variable or aws.java.v1.printLocation system property to 'true'.
This message can be disabled by setting the AWS_JAVA_V1_DISABLE_DEPRECATION_ANNOUNCEMENT environment variable or aws.java.v1.disableDeprecationAnnouncement system property to 'true'.
The AWS SDK for Java 1.x is being used here:
at java.base/java.lang.Thread.getStackTrace(Thread.java:1619)
at com.amazonaws.util.VersionInfoUtils.printDeprecationAn

📊 Aantal records: 10,000
root
 |-- aanhangwagen_autonoom_geremd: string (nullable = true)
 |-- aanhangwagen_middenas_geremd: string (nullable = true)
 |-- aantal_cilinders: string (nullable = true)
 |-- aantal_deuren: string (nullable = true)
 |-- aantal_rolstoelplaatsen: string (nullable = true)
 |-- aantal_staanplaatsen: string (nullable = true)
 |-- aantal_wielen: string (nullable = true)
 |-- aantal_zitplaatsen: string (nullable = true)
 |-- afstand_hart_koppeling_tot_achterzijde_voertuig: string (nullable = true)
 |-- afstand_voorzijde_voertuig_tot_hart_koppeling: string (nullable = true)
 |-- afwijkende_maximum_snelheid: string (nullable = true)
 |-- api_gekentekende_voertuigen_assen: string (nullable = true)
 |-- api_gekentekende_voertuigen_brandstof: string (nullable = true)
 |-- api_gekentekende_voertuigen_carrosserie: string (nullable = true)
 |-- api_gekentekende_voertuigen_carrosserie_specifiek: string (nullable = true)
 |-- api_gekentekende_voertuigen_voertuigklasse: strin

+----------------------------+----------------------------+----------------+-------------+-----------------------+--------------------+-------------+------------------+-----------------------------------------------+---------------------------------------------+---------------------------+-----------------------------------------------+-----------------------------------------------+-----------------------------------------------+-------------------------------------------------+-----------------------------------------------+-------+------------------------+------------------------+---------+--------------+--------------+-----------------------------------+----------------------------------------+-------------------------------------------+----------------------+-------------------------+--------------------+-----------------------+------------+---------------------------------------+--------------------------+-------------------------------------+----------------+---------------+----

#### 03 - Ingest into bronze table

In [5]:
# ======================================================================
# 🔄 Ingest van Landingzone → Bronze (Iceberg via Polaris)
# ======================================================================

bucket = "warehouse"
prefix = "landing"
local_file = find_data_file("gekentekendevoertuigen_sample.json")
object_key = f"{prefix}/{local_file.name}"
s3_uri = f"s3a://{bucket}/{object_key}"

namespace = "bronze"
table_name = "gekentekendevoertuigen"

ns_fqn = namespace
table_fqn = f"{namespace}.{table_name}"

print(f"📥 Lezen vanuit landingzone: {s3_uri}")

# 0) Quick sanity: waar zitten we?
spark.sql("SELECT current_catalog()").show(truncate=False)
spark.sql("SHOW CATALOGS").show(truncate=False)

# 1) Data inlezen uit landingzone
df = spark.read.option("multiline", "true").json(s3_uri)

print(f"📦 Aantal records geladen: {df.count():,}")
df.printSchema()

# 2) Namespace garanderen (in default catalog = polaris)
spark.sql(f"CREATE NAMESPACE IF NOT EXISTS {ns_fqn}")

# 3) Wegschrijven naar Iceberg Bronze (create or replace)
print(f"🧊 Schrijven naar Bronze tabel: {table_fqn}")

(
    df.writeTo(table_fqn)
      .using("iceberg")
      .option("format-version", "2")
      .createOrReplace()
)

# 4) Refresh metadata (optioneel)
spark.catalog.refreshTable(table_fqn)

print(f"✅ Bronze tabel bijgewerkt: {table_fqn}")

# 5) Tabellen tonen
print(f"📋 Tabellen in {ns_fqn}:")
spark.sql(f"SHOW TABLES IN {ns_fqn}").show(truncate=False)

# 6) Bronze teruglezen ter controle
bronze_df = spark.read.table(table_fqn)

print(f"🔁 Records in Bronze: {bronze_df.count():,}")
bronze_df.show(5, truncate=False)


📥 Lezen vanuit landingzone: s3a://warehouse/landing/gekentekendevoertuigen_sample.json
+-----------------+
|current_catalog()|
+-----------------+
|polaris          |
+-----------------+

+-------------+
|catalog      |
+-------------+
|polaris      |
|spark_catalog|
+-------------+

📦 Aantal records geladen: 10,000
root
 |-- aanhangwagen_autonoom_geremd: string (nullable = true)
 |-- aanhangwagen_middenas_geremd: string (nullable = true)
 |-- aantal_cilinders: string (nullable = true)
 |-- aantal_deuren: string (nullable = true)
 |-- aantal_rolstoelplaatsen: string (nullable = true)
 |-- aantal_staanplaatsen: string (nullable = true)
 |-- aantal_wielen: string (nullable = true)
 |-- aantal_zitplaatsen: string (nullable = true)
 |-- afstand_hart_koppeling_tot_achterzijde_voertuig: string (nullable = true)
 |-- afstand_voorzijde_voertuig_tot_hart_koppeling: string (nullable = true)
 |-- afwijkende_maximum_snelheid: string (nullable = true)
 |-- api_gekentekende_voertuigen_assen: string 

26/01/07 12:30:07 ERROR AppendDataExec: Data source write support IcebergBatchWrite(table=polaris.bronze.gekentekendevoertuigen, format=PARQUET) is aborting.
26/01/07 12:30:07 WARN SparkWrite: Skipping cleanup of written files
26/01/07 12:30:07 ERROR AppendDataExec: Data source write support IcebergBatchWrite(table=polaris.bronze.gekentekendevoertuigen, format=PARQUET) aborted.
26/01/07 12:30:07 ERROR Utils: Aborting task
software.amazon.awssdk.core.exception.SdkClientException: Unable to load region from any of the providers in the chain software.amazon.awssdk.regions.providers.DefaultAwsRegionProviderChain@10324202: [software.amazon.awssdk.regions.providers.SystemSettingsRegionProvider@6d124329: Unable to load region from system settings. Region must be specified either via environment variable (AWS_REGION) or  system property (aws.region)., software.amazon.awssdk.regions.providers.AwsProfileRegionProvider@22ec2ac8: No region provided in profile: default, software.amazon.awssdk.regio

Py4JJavaError: An error occurred while calling o139.createOrReplace.
: software.amazon.awssdk.core.exception.SdkClientException: Unable to load region from any of the providers in the chain software.amazon.awssdk.regions.providers.DefaultAwsRegionProviderChain@10324202: [software.amazon.awssdk.regions.providers.SystemSettingsRegionProvider@6d124329: Unable to load region from system settings. Region must be specified either via environment variable (AWS_REGION) or  system property (aws.region)., software.amazon.awssdk.regions.providers.AwsProfileRegionProvider@22ec2ac8: No region provided in profile: default, software.amazon.awssdk.regions.providers.InstanceProfileRegionProvider@35886e97: Unable to contact EC2 metadata service.]
	at software.amazon.awssdk.core.exception.SdkClientException$BuilderImpl.build(SdkClientException.java:130)
	at software.amazon.awssdk.regions.providers.AwsRegionProviderChain.getRegion(AwsRegionProviderChain.java:70)
	at software.amazon.awssdk.awscore.client.b

#### Query de bronze table

In [22]:
from pyspark.sql.functions import col

df = spark.read.table("polaris.bronze.gekentekendevoertuigen")

print("🚗 Top 5 voertuigsoorten:")
(
    df.groupBy("voertuigsoort")
      .count()
      .orderBy(col("count").desc())
      .show(5, truncate=False)
)

print("
🏷️ Top 5 merken:")
(
    df.groupBy("merk")
      .count()
      .orderBy(col("count").desc())
      .show(5, truncate=False)
)

print("
🔤 Top 5 handelsbenamingen:")
(
    df.groupBy("handelsbenaming")
      .count()
      .orderBy(col("count").desc())
      .show(5, truncate=False)
)

print("
⚡ Top 5 voertuigen op vermogen (massarijklaar):")
(
    df.select("merk", "handelsbenaming", "vermogen_massarijklaar")
      .orderBy(col("vermogen_massarijklaar").desc_nulls_last())
      .show(5, truncate=False)
)



🚗 Top 5 voertuigsoorten:
+--------------------+-----+
|voertuigsoort       |count|
+--------------------+-----+
|Personenauto        |7078 |
|Bedrijfsauto        |1237 |
|Bromfiets           |782  |
|Motorfiets          |258  |
|Middenasaanhangwagen|136  |
+--------------------+-----+
only showing top 5 rows


🏷️ Top 5 merken:
+-------------+-----+
|merk         |count|
+-------------+-----+
|VOLKSWAGEN   |1076 |
|PEUGEOT      |615  |
|RENAULT      |606  |
|MERCEDES-BENZ|565  |
|FORD         |553  |
+-------------+-----+
only showing top 5 rows


🔤 Top 5 handelsbenamingen:
+---------------+-----+
|handelsbenaming|count|
+---------------+-----+
|POLO           |219  |
|GOLF           |202  |
|FOCUS          |138  |
|N/A            |135  |
|CLIO           |125  |
+---------------+-----+
only showing top 5 rows


⚡ Top 5 voertuigen op vermogen (massarijklaar):
+-------+--------------------+----------------------+
|merk   |handelsbenaming     |vermogen_massarijklaar|
+-------+-------------